In [ ]:
#5b3ce3597851110001cf6248c7ae45297d34493ea117f6002af2b8f1
origin = (-33.865143, 151.209900)  #Sydney
destination1 = (-33.4267, 151.3417) # Gosford
destination2 = (-32.9283, 151.7817)  #Newcastle
destination = (-27.4705, 153.0260) #Brisbane


In [ ]:
import requests
import folium

def get_route_details(origin, destination, api_key):
    """
    Get route details including coordinates and distance from origin to destination using OpenRouteService API.

    Args:
    - origin (tuple): Latitude and longitude of the origin point (e.g., (latitude, longitude)).
    - destination (tuple): Latitude and longitude of the destination point (e.g., (latitude, longitude)).
    - api_key (str): Your OpenRouteService API key.

    Returns:
    - route_coords (list): List of tuples containing the route coordinates.
    - distance (float): Distance in kilometers.
    """
    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    params = {
        "api_key": api_key,
        "start": f"{origin[1]},{origin[0]}",  # Note the order of coordinates (longitude, latitude).
        "end": f"{destination[1]},{destination[0]}"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        route_data = response.json()
        route_coords = []
        for point in route_data['features'][0]['geometry']['coordinates']:
            route_coords.append((point[1], point[0]))  # Reversing the order to (latitude, longitude)
        distance = route_data['features'][0]['properties']['summary']['distance'] / 1000  # Convert meters to kilometers
        return route_coords, distance
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None, None

# Example usage
origin = (-33.865143, 151.209900)  #Sydney
destination = (-31.407190, 152.900783) #Brisbane
api_key = "5b3ce3597851110001cf6248c7ae45297d34493ea117f6002af2b8f1"  # Replace this with your actual OpenRouteService API key

route_coords, distance = get_route_details(origin, destination, api_key)
if route_coords and distance is not None:
    print(f"Route coordinates retrieved successfully. Distance: {distance:.2f} km")
    # Plotting the route on an actual map
    map_obj = folium.Map(location=origin, zoom_start=6)
    folium.PolyLine(locations=route_coords, color='blue').add_to(map_obj)
    folium.Marker(location=origin, popup='Origin').add_to(map_obj)
    folium.Marker(location=destination, popup='Destination').add_to(map_obj)
    display(map_obj)
else:
    print("Failed to retrieve the route coordinates or distance.")


Route coordinates retrieved successfully. Distance: 395.16 km


In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(origin, destination):
    """
    Calculate the straight-line distance between two points on the Earth's surface using the Haversine formula.

    Args:
    - origin (tuple): Latitude and longitude of the origin point in degrees (e.g., (latitude, longitude)).
    - destination (tuple): Latitude and longitude of the destination point in degrees (e.g., (latitude, longitude)).

    Returns:
    - distance (float): Straight-line distance between the two points in kilometers.
    """
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = radians(origin[0]), radians(origin[1])
    lat2, lon2 = radians(destination[0]), radians(destination[1])

    # Calculate differences in latitude and longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = R * c

    return distance

# Example usage
origin = (-33.865143, 151.209900)  #Sydney
destination = (-32.9283, 151.7817)  #Newcastle

distance = haversine_distance(origin, destination)
print(f"Straight-line distance between origin and destination: {distance:.2f} km")


Straight-line distance between origin and destination: 116.92 km


In [ ]:
import requests
import folium

def get_route(origin, destination, api_key):
    """
    Get route details including coordinates and distance from origin to destination using OpenRouteService API.

    Args:
    - origin (tuple): Latitude and longitude of the origin point (e.g., (latitude, longitude)).
    - destination (tuple): Latitude and longitude of the destination point (e.g., (latitude, longitude)).
    - api_key (str): Your OpenRouteService API key.

    Returns:
    - route_coords (list): List of tuples containing the route coordinates.
    - distance (float): Distance in kilometers.
    """
    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    params = {
        "api_key": api_key,
        "start": f"{origin[1]},{origin[0]}",  # Note the order of coordinates (longitude, latitude).
        "end": f"{destination[1]},{destination[0]}"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        route_data = response.json()
        route_coords = []
        for point in route_data['features'][0]['geometry']['coordinates']:
            route_coords.append((point[1], point[0]))  # Reversing the order to (latitude, longitude)
        distance = route_data['features'][0]['properties']['summary']['distance'] / 1000  # Convert meters to kilometers
        return route_coords, distance
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None, None

def join_journeys(origin, destination1, destination2, api_key):
    """
    Join two journeys together: first from origin to destination1, then from destination1 to destination2.

    Args:
    - origin (tuple): Latitude and longitude of the origin point (e.g., (latitude, longitude)).
    - destination1 (tuple): Latitude and longitude of the first destination point (e.g., (latitude, longitude)).
    - destination2 (tuple): Latitude and longitude of the second destination point (e.g., (latitude, longitude)).
    - api_key (str): Your OpenRouteService API key.

    Returns:
    - route_coords (list): List of tuples containing the route coordinates.
    - distance (float): Distance in kilometers.
    """
    # Calculate route from origin to destination1
    route_coords1, distance1 = get_route(origin, destination1, api_key)
    if route_coords1 is None:
        print("Failed to retrieve the first route.")
        return None, None

    # Calculate route from destination1 to destination2
    route_coords2, distance2 = get_route(destination1, destination2, api_key)
    if route_coords2 is None:
        print("Failed to retrieve the second route.")
        return None, None

    # Combine the routes
    route_coords = route_coords1 + route_coords2
    distance = distance1 + distance2

    return route_coords, distance

# Example usage
origin = (-33.865143, 151.209900)  #Sydney
destination1 = (-31.407190, 152.900783) # Gosford
destination2 = (-27.4705, 153.0260) #Brisbane
api_key = "5b3ce3597851110001cf6248c7ae45297d34493ea117f6002af2b8f1"  # Replace this with your actual OpenRouteService API key

route_coords, distance = join_journeys(origin, destination1, destination2, api_key)
if route_coords and distance is not None:
    print(f"Joined route retrieved successfully. Total Distance: {distance:.2f} km")
    # Plotting the route on an actual map
    map_obj = folium.Map(location=origin, zoom_start=6)
    folium.PolyLine(locations=route_coords, color='blue').add_to(map_obj)
    folium.Marker(location=origin, popup='Origin').add_to(map_obj)
    folium.Marker(location=destination1, popup='Destination 1').add_to(map_obj)
    folium.Marker(location=destination2, popup='Destination 2').add_to(map_obj)
    display(map_obj)
else:
    print("Failed to retrieve the joined route.")


Joined route retrieved successfully. Total Distance: 934.76 km


In [ ]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(origin, destination):
    """
    Calculate the straight-line distance between two points on the Earth's surface using the Haversine formula.

    Args:
    - origin (tuple): Latitude and longitude of the origin point in degrees (e.g., (latitude, longitude)).
    - destination (tuple): Latitude and longitude of the destination point in degrees (e.g., (latitude, longitude)).

    Returns:
    - distance (float): Straight-line distance between the two points in kilometers.
    """
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = radians(origin[0]), radians(origin[1])
    lat2, lon2 = radians(destination[0]), radians(destination[1])

    # Calculate differences in latitude and longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = R * c

    return distance

# Example DataFrame
data = {
    'latitude': [52.520008, 48.856613, 51.5074],
    'longitude': [13.404954, 2.352222, 0.1278]
}
df = pd.DataFrame(data)

# Calculate distances and create DataFrame
distances_list = []
for i in range(len(df)):
    for j in range(len(df)):
        if i != j:
            origin = (df['latitude'][i], df['longitude'][i])
            destination = (df['latitude'][j], df['longitude'][j])
            distance = haversine_distance(origin, destination)
            distances_list.append({'origin': i, 'destination': j, 'distance': distance})

# Create DataFrame from distances list
distances_df = pd.DataFrame(distances_list)
print(distances_df)


   origin  destination    distance
0       0            1  877.458688
1       0            2  914.279732
2       1            0  877.458688
3       1            2  334.575595
4       2            0  914.279732
5       2            1  334.575595
